In [ ]:
#command : yolo task=detect mode=train model=yolov8s.pt imgsz=1280 data="C:\Users\maeva\OneDrive\Bureau\yolov8\pyronear\yolo_models\albumentations\data.yaml" epochs=10 batch=64 name=yolov10n

This is a script that verify that the custom augmentations are being picked up during training. We have to modify the training script to read and apply these augmentations.

In [12]:
import albumentations as A
import cv2
import yaml
from ultralytics.data.augment import Albumentations

# Define the CustomAlbumentations class
class CustomAlbumentations(Albumentations):
    def __init__(self, yaml_file='data.yaml', p=1.0):
        self.p = p
        self.transform = None
        try:
            # Load custom augmentations from the YAML file
            with open(yaml_file, 'r') as f:
                data = yaml.safe_load(f)
            
            # Check if 'augmentations' key is in the YAML file
            if 'augmentations' not in data:
                print(f"Warning: 'augmentations' key not found in {yaml_file}.")
                return
            
            custom_augmentations = data.get('augmentations', [])
            
            # Initialize albumentations transformations
            T = []
            for aug in custom_augmentations:
                try:
                    aug_type = getattr(A, aug['type'])
                    params = {k: v for k, v in aug.items() if k != 'type'}
                    
                    # Print parameters for debugging
                    print(f"Augmentation type: {aug['type']}, parameters: {params}")
                    
                    T.append(aug_type(**params))
                except AttributeError as e:
                    print(f"Error: {aug['type']} not found in albumentations. {e}")
                except Exception as e:
                    print(f"Error processing augmentation {aug}. {e}")
            
            if T:
                # Adjust bbox_params only if using bounding boxes
                self.transform = A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))
            else:
                print("No valid augmentations found.")
        
        except ImportError:
            print("Albumentations library is not installed.")
        except yaml.YAMLError as e:
            print(f"Error reading YAML file: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

    def apply(self, image, bboxes=None):
        bboxes = bboxes or []
        if self.transform:
            try:
                augmented = self.transform(image=image, bboxes=bboxes)
                return augmented['image'], augmented['bboxes']
            except ValueError as e:
                print(f"ValueError during augmentation: {e}")
                return image, bboxes
            except Exception as e:
                print(f"Error applying transformations: {e}")
                return image, bboxes
        return image, bboxes

# Function to apply augmentations and save the output image
def test_augmentations(image_path, albumentations):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    
    # Define bounding boxes (empty list if not used)
    bboxes = []

    # Apply augmentations
    augmented_image, _ = albumentations.apply(image, bboxes)
    
    # Convert back to BGR for saving
    augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
    
    # Save augmented image
    output_path = 'augmented_image.jpg'
    cv2.imwrite(output_path, augmented_image)
    print(f"Augmented image saved at {output_path}")

# Main function to test
if __name__ == "__main__":
    # Initialize CustomAlbumentations with the path to your YAML file
    albumentations = CustomAlbumentations(yaml_file='data.yaml')
    
    # Test with a sample image
    test_augmentations('image_test.jpg', albumentations)


Augmentation type: Blur, parameters: {'p': 0.01, 'blur_limit': 7}
Augmentation type: RandomBrightnessContrast, parameters: {'p': 0.5, 'brightness_limit': 0.1, 'contrast_limit': 0.1}
Augmentation type: Crop, parameters: {'p': 0.5, 'height': 1024, 'width': 1024}
Augmentation type: GaussNoise, parameters: {'p': 0.5, 'var_limit': [10.0, 50.0]}
Augmentation type: HorizontalFlip, parameters: {'p': 0.5}
Augmentation type: HueSaturationValue, parameters: {'p': 0.5, 'hue_shift_limit': 20, 'sat_shift_limit': 50}
Augmentation type: MotionBlur, parameters: {'p': 0.5, 'blur_limit': 7}
ValueError during augmentation: Your 'label_fields' are not valid - them must have same names as params in dict
Augmented image saved at augmented_image.jpg


C:\Users\maeva\AppData\Local\Temp\ipykernel_27844\3660001763.py:33: UserWarning: Argument 'height' is not valid and will be ignored.
  T.append(aug_type(**params))
C:\Users\maeva\AppData\Local\Temp\ipykernel_27844\3660001763.py:33: UserWarning: Argument 'width' is not valid and will be ignored.
  T.append(aug_type(**params))
